In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.utils import make_grid
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [38]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='/mnist_data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='/mnist_data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=20, shuffle=True)
test_loader = DataLoader(test_data, batch_size=20, shuffle=False)

In [21]:
class ConvNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = f.relu(self.conv1(X))
    X = f.max_pool2d(X, 2, 2)
    X = f.relu(self.conv2(X))
    X = f.max_pool2d(X, 2, 2)

    X = X.view(-1, 16*5*5)
    X = f.relu(self.fc1(X))
    X = f.relu(self.fc2(X))
    X = self.fc3(X)

    return f.log_softmax(X, dim=1)

In [34]:
torch.manual_seed(30)
model = ConvNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

epochs = 3
losses = []
accuracy = []

for epoch in range(epochs):
  correct = 0
  size = 0
  for b, (X_train, y_train) in enumerate(train_loader):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    predicted = torch.max(y_pred.data, 1)[1]

    #amount of correct predictions in current batch
    batch_corr = (predicted == y_train).sum()
    correct += batch_corr
    size += int(y_train.size()[0])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if b%800 == 0:
      print(f'Epoch {epoch}, batch number {b} loss {loss}')

  losses.append(loss)
  accuracy.append(correct/size)

print(f'losses: {losses}')
print(f'acc: {accuracy}')

Epoch 0, batch number 0 loss 2.303067445755005
Epoch 0, batch number 800 loss 0.15863728523254395
Epoch 0, batch number 1600 loss 0.10212469846010208
Epoch 0, batch number 2400 loss 0.1581864207983017
Epoch 0, batch number 3200 loss 0.015582998283207417
Epoch 0, batch number 4000 loss 0.013652809895575047
Epoch 0, batch number 4800 loss 0.046348389238119125
Epoch 0, batch number 5600 loss 0.14677436649799347
Epoch 0, batch number 6400 loss 0.22526387870311737
Epoch 0, batch number 7200 loss 0.013300127349793911
Epoch 0, batch number 8000 loss 0.018726753070950508
Epoch 1, batch number 0 loss 0.031052665784955025
Epoch 1, batch number 800 loss 0.033464618027210236
Epoch 1, batch number 1600 loss 0.009655318222939968
Epoch 1, batch number 2400 loss 0.043690502643585205
Epoch 1, batch number 3200 loss 0.0014886591816321015
Epoch 1, batch number 4000 loss 0.14464183151721954
Epoch 1, batch number 4800 loss 0.020114904269576073
Epoch 1, batch number 5600 loss 0.001312742941081524
Epoch 1, b

In [40]:
with torch.no_grad():
  correct = 0
  size = 0
  for b, (X_test, y_test) in enumerate(test_loader):
    y_pred = model(X_test)
    predicted = torch.max(y_pred.data, 1)[1]
    batch_corr = (predicted == y_test).sum()
    correct += batch_corr
    size += int(y_test.size()[0]) #batch size

accuracy = correct/size
print(accuracy)

tensor(0.9857)
